# Overview

This notebook is the development notes for using `bluesky` and `apstools` to conduct Tomography experiment at 6BM-A at the advanced photon source.

The notebook contains the following sections:
* preparation
    * installation and configuration of `bluesky`+`apstools` for 6BM-A
        * package installation
        * meta-data broker configuration
    * user-define functions that is useful for the tomography experiment
* device declaration: declare `ophyd` interface for `Epics` devices
    * device initilization
* tomography experiment
    * plan generation
    * dry-run
    * connect to devices for experiment

# Preparation

Install `bluesky` and `apstools` with the following command:

```bash
pip install -U pip
pip install boltons mongoquery pims pyepics pyRestTable tzlocal jupyter suitcase matplotlib
pip install git+https://github.com/Nikea/historydict#egg=historydict \
            git+https://github.com/NSLS-II/amostra#egg=amostra \
            git+https://github.com/NSLS-II/bluesky#egg=bluesky \
            git+https://github.com/NSLS-II/databroker#egg=databroker \
            git+https://github.com/NSLS-II/doct#egg=doct \
            git+https://github.com/NSLS-II/event-model#egg=event_model \
            git+https://github.com/NSLS-II/ophyd#egg=ophyd \
            git+https://github.com/NSLS-II/hklpy#egg=hklpy
pip install apstools
```

Then configure MongoDB for meta-data handling

The following YAML config file should be place under `~/.config/` to use MongoDB as meta-data handler.

```yml
# ~/.config/databroker/mongodb_config.yml

description: 'heavyweight shared database'
metadatastore:
   module: 'databroker.headersource.mongo'
   class: 'MDS'
   config:
       host: 'MONGODB_HOST_NAME'
       port: 27017
       database: 'metadatastore-production-v1'
       timezone: 'US/Central'
assets:
   module: 'databroker.assets.mongo'
   class: 'Registry'
   config:
       host: 'MONGODB_HOST_NAME'
       port: 27017
       database: 'filestore-production-v1'
```

In [ ]:
# initialize the data base
from databroker import Broker
db = Broker.named("mongodb_config")

# subscribe both mongodb and callback to runtime engine
import bluesky
from bluesky import RunEngine
from bluesky.callbacks.best_effort import BestEffortCallback

RE = RunEngine({})
RE.subscribe(db.insert)
RE.subscribe(BestEffortCallback())

In [ ]:
# user defined function goes here

# 1. checking the APS beam status
import apstools.devices as APS_devices

aps = APS_devices.ApsMachineParametersDevice(name="APS")

Now from the hardware end, we need additional setup IOC (soft) for this status, recommended method `VME`

Here is a quick example from 2BM
```python
instrument_in_use = EpicsSignalRO(
    "2bm:instrument_in_use", 
    name="instrument_in_use")

def operations_in_2bmb():
    """returns True if allowed to use X-ray beam in 2-BM-B station"""
    v = instrument_in_use.value
    enums = instrument_in_use.enum_strs
    return enums[v] == "2-BM-B"
```

In [ ]:
# 2. assume we have the same setup describe above

from ophyd import EpicsSignalRO

instrument_in_use = EpicsSignalRO("6bm:instrument_in_use", name="instrument_in_use")

In [ ]:
# 3. check ambien light in hutch
import apstools.synApps_ophyd

# grab all calcs
calcs = apstools.synApps_ophyd.userCalcsDevice("6bma1:", name="calcs")

# 2 is ambient light checker
calc2 = clacs.calc2

In [ ]:
# 4. define the ENV VAR denote the experiment nature
# NOTE:
#    the return value of instrument_in_use is not yet decided

# conducting experiment mode
in_production = aps.inUserOperations and (instrument_in_use.value>0) and (calc2<0)

# testing mode, supercede in_production
in_dryrun = True

# Device declaration

In [1]:
# necessary import
import apstools.devices as APS_devices

## shutter

In [ ]:
if in_production:
    # define the real shutter used at 6BMA@APS
    # NOTE: 
    #   this requires connection to the hardware, otherwise a connection error will be raised

    A_shutter = APS_devices.ApsPssShutterWithStatus(
            "6bmb1:rShtrA",
            "PA:06BM:STA_A_FES_OPEN_PL",
            name="A_shutter",
        )

    A_shutter.summary()
    
    # no scans until A_shutter is open
    suspend_A_shutter = bluesky.suspenders.SuspendFloor(A_shutter.pss_state, 1)
    #suspend_A_shutter.install(RE)
    RE.install_suspender(suspend_A_shutter)
    
    # no scans if aps.current is too low
    suspend_APS_current = bluesky.suspenders.SuspendFloor(aps.current, 2, resume_thresh=10)
    RE.install_suspender(suspend_APS_current)

else:
    # first, a simulated shutter to demonstrate the design of epics controled shutter

    A_shutter = APS_devices.SimulatedApsPssShutterWithStatus(name="A_shutter")

    A_shutter.summary()

To open a shutter, use `A_shutter.open`. 
Similarly, the shutter can be closed with `A_shutter.close`.
One can also check the current status of the shutter using `A_shutter.isOpen`.

## area detector

In [13]:
import os
import datetime

# production control ENV vars
ADPV_prefix = "1idPG3"  # AreaDetector prefix

# -----------------------
# -- user config block --
# -----------------------
USER = "enyaw_ecurb"
PROEJCT = "redqueen"     # Project name
EXP = "bluerose"         # Experiment name
OUTPUT_ROOT = "/tmp"     # The root for output data storage
FILE_PATH =  os.path.join(OUTPUT_ROOT, USER, PROEJCT, str(datetime.date.today()), " ")
FILE_PREFIX = f"{PROEJCT}_{EXP}"

# show where files will be stored
print(FILE_PATH)
print(FILE_PREFIX)

/tmp/enyaw_ecurb/redqueen/2019-04-05/ 
redqueen_bluerose


In [ ]:
# As of today, there is still no support for Python class enclosure, so we have to make a class for the detector
from ophyd import AreaDetector
from ophyd import SingleTrigger
from ophyd import ADComponent
from ophyd import PointGreyDetectorCam
from ophyd import ProcessPlugin
from ophyd import TIFFPlugin
from ophyd import sim

class PointGreyDetectorCam6BM(PointGreyDetectorCam):
    """Extend the default Point Grey detector cam"""
    frame_rate_on_off = ADComponent(EpicsSignalWithRBV, "FrameRateOnOff")


class PointGreyDetector6BM(SingleTrigger, AreaDetector):
    """Point Gray area detector used at 6BM"""
    # cam component
    cam = ADComponent(PointGreyDetectorCam6BM, "cam1:")
    
    # proc plugin
    proc1 = ADComponent(ProcessPlugin, suffix="Proc1:")
    
    # tiff plugin
    tiff1 = ADComponent(
        TIFFPlugin,
        suffix="TIFF1:",
        root="/",                               # for databroker
        write_path_template=FILE_PATH,          # for EPICS AD
    )
    

# ------------------------------------
# ----- Instantiate the detector ----- 
# ------------------------------------

# Area Detector (AD) config block
config_cam = {
    "num_images":     1,           # number of images (nFrame)
    "image_mode":     "Multiple",  #
    "trigger_mode":   "Internal",  #
    "acquire_time":   1,           # exposure time (fExposureTime)
    "acquire_period": 1+0.01,      #
    "gain":           5,           # detector gain [0~30]
}

config_proc1 = {
    "enable_callbacks": 1,  # toggle on proc1
    "enable_filter":    1,  # enable filter
    "num_filter":       1,  # change number_filtered in proc1 (same as nFrame)
    "reset_filter":     1,  # reset number_filtered
}

config_tiff1 = {
    "nd_array_port":    "PROC1",      # switch port for TIFF plugin
    "file_write_mode":  "Stream",     # change write mode
    "auto_save":        "Yes",        # turn on file save
    "file_path":        FILE_PATH,    # set file path
    "file_name":        FILE_PREFIX,  # img name prefix
}


if in_production or in_dryrun:
    pg3_det = PointGreyDetector6BM("1idPG3", name='pg3_det')
    
    # catch timeout error in case detector not responding
    try:
        for k, v in config_cam.items():     pg3_det.cam.stage_sigs[k]   = v
        for k, v in config_proc1.items():   pg3_det.proc1.stage_sigs[k] = v
        for k, v in config_tiff1.items():   pg3_det.tiff1.stage_sigs[k] = v
    except TimeoutError as _exc:
        print(f"{_exc}\n !! Could not connect with area detector {pg3_det}")
else:
    pg3_det = sim.noisy_det  # use ophyd simulated detector

## motor

In [ ]:
from ophyd import MotorBundle

# NOTE: 
#    the PV for actual motors is still unknown
class TomoStage(MotorBundle):
    #rotation
    preci = Component(EpicsMotor, "6bmpreci:m1",
                      name='preci',
                     )
    
    samX = Component(EpicsMotor, "???",
                     name='samX',
                    )
    
    samY = Component(EpicsMotor, "???",
                     name="samY",
                    )

if in_production or in_dryrun:
    tomostage = TomoStage()

    samx  = tomostage.samX
    samy  = tomostage.samY
    preci = tomostage.preci

else:
    tomostage = sim.motor

# Tomography experiment

## Scan plans

Define the scan parameters within plan functions

In [ ]:
import bluesky.plans as bp
import bluesky.plan_stubs as bps


# ---------------------------
# ----- Scan parameters -----
# ---------------------------
n_white    = 10
n_dark     = 10
samOutDist = -4.00                           # mm
omega_step = 0.25                            # degrees
number_of_projections = int(180/omega_step)  # 0 ~ 180
time_wait  = 0.01                            # s


def collect_white_field():
    """collect 10 white field images before and after tomo scan"""
    current_samx = samx.position

    yield from bps.mv(
        A_shutter,              "open", 
        samx,                   current_samx+samOutDist,
        det.cam.trigger_mode,   "Internal",
        det.cam.num_images,     n_white,
        det.proc1.reset_filter, 1,
    )
    
    yield from bps.trigger(det)
    
    yield from lambda _: while det.tiff1.num_captured.value < n_white: yield from bps.sleep(time_wait)

    yield from bps.mv(
        samx, current_samx
    )


def collect_dark_field():
    """collect 10 dark field images after tomo scan"""
    current_samx = samx.position
    
    yield from bps.mv(
        A_shutter,              "close", 
        samx,                   current_samx+samOutDist,
        det.cam.trigger_mode,   "Internal",
        det.cam.num_images,     n_dark,
        det.proc1.reset_filter, 1,
    )
    
    yield from bps.trigger(pg3_det)
    
    yield from lambda _: while det.tiff1.num_captured.value < n_dark: yield from bps.sleep(time_wait)

    yield from bps.mv(
        samx, current_samx
    )
    
    
def collect_projection():
    """tomo scna"""
    yield from bp.scan(
        [pg3_det], 
        preci, 0, 180, number_of_projections,
    )

## Scan plan pruning prior to execution

Use the `summarize_plan` feature from bluesky to perform sanity check on the tomo scan plan

In [ ]:
from bluesky.simulators import summarize_plan

summarize_plan(collect_white_field)

summarize_plan(collect_dark_field)

summarize_plan(collect_projection)

## Excuete plans

In [ ]:
RE(collect_white_field)
RE(collect_projection)
RE(collect_white_field)
RE(collect_dark_field)